# Test case multi-ecehlon inventory control program
Welcome to this test case of the multi-echelon model implemented within the scope of the Master Thesis called "Comparing single- and multi-echelon methods for inventory control of spare parts at Volvo".

Authors of this project is Jakob Bengtsson and Alexander Larsson, supervisors were Christian Beckers and Johan Lidvall.

This test case displays how to use to program in order to generate optimal reorder-points for an (R,Q) - policy for the example item present in the South African network.

A walkthrough of this test case can give you a brief understanding of the setup of the program and how pieces are connected. However, to really understand what's going on we refer to the theory section (Section 4) of the associated thesis report.


In [1]:
import pandas as pd
import numpy as np

from src.warehouse_modeling.induced_backorder_cost import *
from src.warehouse_modeling.lead_time_approximation import *
from src.warehouse_modeling.warehouse_optimization import *
from src.warehouse_modeling.warehouse_demand_modeling import *

from src.single_echelon_utils.inventory_level_computation import *
from src.single_echelon_utils.service_level_computation import *
from src.single_echelon_utils.dealer_optimization import *

from utils import *



## Initating inputs
A test case is submitted in the directory "Test_Case". If you have other cases to try out, please enter them here. Remark that even if demand size data is not used (which is the case if demand distribution is Normal, Poisson or NBD) this input is still required by the program. A workaround is to put a placeholder file with demand size of 1 haing probability 100 % as this will not have any effect on the results.

The capital cost used is in percentage per day. The value of 0.15 was given by the supervisors as the holding cost rate in the South African network.

In [5]:
indata_path = "test_data/test_case_indata.csv"
indata_demand_size_dist_path = "test_data/test_case_indata_demand_sizes.csv"
outdata_path = "test_data/test_case_outdata.csv"
capital_cost = 0.15/365

Inputs are handled and converted to arrays in the cell below. Note that there are several columns required to exist in the indata file.

In [6]:
indata_df = pd.read_csv(indata_path)

outdata_df = indata_df.copy()

# Ensure correct columns are present:
required_columns = {"Installation id", "Type", "Name", "Transport time", "Q", 
    "Unit cost", "Target item fill rate", "Demand distribution",	"Demand mean per time unit", "Demand stdev per time unit"}
    
if not required_columns.issubset(set(indata_df.columns.to_list())):
    raise ValueError("Indata doesn't contain all required fields, see documentation.")

# Retrieving the data about dealers.
Q_dealer_arr = indata_df.get(indata_df["Type"] == "Dealer").get("Q").to_numpy().astype("int32")
mu_dealer_arr = indata_df.get(indata_df["Type"] == "Dealer").get("Demand mean per time unit").to_numpy().astype("float64")
demand_type_arr = indata_df.get(indata_df["Type"] == "Dealer").get("Demand distribution").to_numpy()
h_dealer_arr = capital_cost * indata_df.get(indata_df["Type"] == "Dealer").get("Unit cost").to_numpy().astype("float64")
fill_rate_target_arr = indata_df.get(indata_df["Type"] == "Dealer").get("Target item fill rate").to_numpy().astype("float64")
l_dealer_arr = indata_df.get(indata_df["Type"] == "Dealer").get("Transport time").to_numpy().astype("float64")

# If demand distribution is poisson, retrieve 
sigma_dealer_list = []
for id in indata_df.get(indata_df["Type"] == "Dealer").get("Installation id"):
    if str(indata_df.get(indata_df["Installation id"] == id).get("Demand distribution")) == "Poisson":
        sigma_dealer_list.append(math.sqrt(
            float(indata_df.get(outdata_df["Installation id"]== id).get("Demand mean per time unit"))))
    else:
        sigma_dealer_list.append(float(indata_df.get(indata_df["Installation id"] == id).get("Demand stdev per time unit")))        
sigma_dealer_arr = np.array(sigma_dealer_list)

# Input compounding distribution arrays here!
# Supposed to have one row per dealer.
if indata_demand_size_dist_path[-3:] == "csv":
    compounding_dist_df = pd.read_csv(indata_demand_size_dist_path)
else:
    compounding_dist_df = pd.read_excel(indata_path,indata_demand_size_dist_path)

compounding_dist_matrix = compounding_dist_df.to_numpy().T[1:] # Each array is a column in excel, transposing and removing first row holding item amounts.
compounding_dist_matrix = np.nan_to_num(compounding_dist_matrix,copy = True)


C:\Users\PCAdmin\AppData\Local\Temp\ipykernel_11800\3085652986.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_dealer_list.append(float(indata_df.get(indata_df["Installation id"] == id).get("Demand stdev per time unit")))


## Regional distribution center demand
The demand at the RDC (or central warehouse in the network) depend on the inventory policies at dealers. 

First, as dealers are only demanding Q units, the warehouse demand during the leadtime can only be integer multiples of the smalles common divisor of the dealers order quantities.


In [8]:
#Find the smallest common divisor of Q.
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
print(f"Smallest common divisor of order quantities is: {Q_subbatch_size}")

Smallest common divisor of order quantities is: 1


In [10]:
   
#Read warehouse values.
L_wh = float(indata_df.get(indata_df["Type"]=="RDC").get("Transport time"))
h_rdc = capital_cost*float(indata_df.get(indata_df["Type"] == "RDC").get("Unit cost"))
Q_0 = int(int(indata_df.get(indata_df["Type"] == "RDC").get("Q"))/Q_subbatch_size) # Observe, Q_0 is in subbatches.

print(f"Leadtime from CDC to RDC is: {L_wh}, holding cost per day at rdc is: {h_rdc} and order quantity is {Q_0}")

Leadtime from CDC to RDC is: 58.0, holding cost per day at rdc is: 0.12380547945205478 and order quantity is 29


C:\Users\PCAdmin\AppData\Local\Temp\ipykernel_11800\1817249328.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  L_wh = float(indata_df.get(indata_df["Type"]=="RDC").get("Transport time"))
C:\Users\PCAdmin\AppData\Local\Temp\ipykernel_11800\1817249328.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  h_rdc = capital_cost*float(indata_df.get(indata_df["Type"] == "RDC").get("Unit cost"))
C:\Users\PCAdmin\AppData\Local\Temp\ipykernel_11800\1817249328.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  Q_0 = int(int(indata_df.get(indata_df["Type"] == "RDC").get("Q"))/Q_subbatch_size) # Observe, Q_0 is in subbatches.


With inputs present and Q_subbatch computed, it is time to find the warehouse demand array. This is done in the function "warehouse_subbatch_demand_probability_array" which you can find in directory "warheouse_modeling". The workings of this is thouroughly described in section 4.10 of the report and detailed references can be found in every part of the function in the "warehouse modeling" directory.

In [12]:
  
# Central warehouse demand
# --------------------------------------------------------------------------
# Computing subbatch demand probability array, distribution, lead time demand mean, and 
# lead time demand variance at central warehouse.
# Observe that these values are returned in "subbatches".
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(
Q_dealer_arr, mu_dealer_arr, sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size, 
compounding_dist_matrix)
outdata_df.loc[outdata_df["Type"] == "RDC","Demand distribution"] = wh_dist
outdata_df.loc[outdata_df["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdata_df.loc[outdata_df["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdata_df.loc[outdata_df["Type"] == "RDC","Demand mean per time unit"] = mu_L * Q_subbatch_size/L_wh
outdata_df.loc[outdata_df["Type"] == "RDC","Demand stdev per time unit"] = math.sqrt(sigma2_L * Q_subbatch_size/L_wh)
 
print(f"""The demand probability is interpreted like this:

Probability of lead time demand = 0*Q_subbatch is {rdc_f_u_probability_array[0]}
Probability of lead time demand = 1*Q_subbatch is {rdc_f_u_probability_array[1]}
Probability of lead time demand = 2*Q_subbatch is {rdc_f_u_probability_array[2]}
...""")

2024-10-15 15:38:12,448 - default_log - DEBUG - new_function
2024-10-15 15:38:12,449 - sparse_log - DEBUG - warehouse_subbatch_demand_probability_array
2024-10-15 15:38:12,450 - default_log - DEBUG - warehouse_demand_mean_approximation
2024-10-15 15:38:12,451 - default_log - DEBUG - warehouse_demand_mean_approximation ended.
2024-10-15 15:38:12,452 - default_log - DEBUG - warehouse_demand_variance_approximation
2024-10-15 15:38:12,452 - default_log - DEBUG - warehouse_demand_variance_term
2024-10-15 15:38:12,453 - default_log - DEBUG - pmf_func_warehouse_subbatch_demand
2024-10-15 15:38:12,985 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand
2024-10-15 15:38:12,986 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand ended.
2024-10-15 15:38:12,987 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand
2024-10-15 15:38:12,987 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand ended.
2024-10-15 15:38:12,988 - default_log - DEBUG

The demand probability is interpreted like this:

Probability of lead time demand = 0*Q_subbatch is 2.1774057322736473e-19
Probability of lead time demand = 1*Q_subbatch is 6.015553201084292e-18
Probability of lead time demand = 2*Q_subbatch is 8.495198857398937e-17
...


## Induced backorder cost
When lead time demand at the warehouse is found. The induced backorder cost is required in order to find optimal reorder points R.

In [13]:
   
# Calculating induced backorder cost.
# --------------------------------------------------------------------------
# Computing shortage costs.
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
    
# Computing induced backorder cost for each retailer.
beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,
mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))
beta_arr = np.array(beta_list)

# Computing weighted backorder cost at central warehouse.
mu_wh = mu_L/L_wh * Q_subbatch_size
beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
    
outdata_df.loc[outdata_df["Type"] == "RDC", "Holding cost"] = h_rdc
outdata_df.loc[outdata_df["Type"] == "Dealer", "Holding cost"] = h_dealer_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Estimated shortage cost"] = p_dealer_arr
outdata_df.loc[outdata_df["Type"] == "RDC", "Beta"] = beta_rdc
outdata_df.loc[outdata_df["Type"] == "Dealer", "Beta"] = beta_arr

print(f"The induced backorder cost at the RDC is: {beta_rdc}")

2024-10-15 15:38:28,909 - default_log - DEBUG - induced_backorder_cost_opt
2024-10-15 15:38:28,911 - default_log - DEBUG - norm_sigma
2024-10-15 15:38:28,912 - default_log - DEBUG - norm_sigma ended.
2024-10-15 15:38:28,912 - default_log - DEBUG - induced_backorder_cost_opt ended.
2024-10-15 15:38:28,913 - default_log - DEBUG - induced_backorder_cost_opt
2024-10-15 15:38:28,914 - default_log - DEBUG - norm_sigma
2024-10-15 15:38:28,914 - default_log - DEBUG - norm_sigma ended.
2024-10-15 15:38:28,915 - default_log - DEBUG - induced_backorder_cost_opt ended.
2024-10-15 15:38:28,916 - default_log - DEBUG - induced_backorder_cost_opt
2024-10-15 15:38:28,917 - default_log - DEBUG - norm_sigma
2024-10-15 15:38:28,918 - default_log - DEBUG - norm_sigma ended.
2024-10-15 15:38:28,919 - default_log - DEBUG - induced_backorder_cost_opt ended.
2024-10-15 15:38:28,920 - default_log - DEBUG - induced_backorder_cost_opt
2024-10-15 15:38:28,921 - default_log - DEBUG - norm_sigma
2024-10-15 15:38:28,

The induced backorder cost at the RDC is: 0.28001674915125674


In order to provide a better understanding of the induced backorder cost we here translate is to a fill rate value.

In [14]:

# Calculating target fill rate corresponding to beta estimate
target_fill_rate_warehouse = beta_rdc/(beta_rdc + h_rdc)
outdata_df.loc[outdata_df["Type"]== "RDC", "Target item fill rate"] = target_fill_rate_warehouse

print(f"The induced backorder cost of {round(beta_rdc,4)} gives a similar reorder point as setting the target fill rate to approximately {100*round(target_fill_rate_warehouse,2)} %")

The induced backorder cost of 0.28 gives a similar reorder point as setting the target fill rate to approximately 69.0 %


## Optimizing reorder point at RDC/central warehouse

The optimal reorder point is computed by balancing costs of holding inventory and costs of backorders. The costs of holding inventory increase with more inventory. Backorder costs increase with less inventory. 

Here, the induced backorder costs is used to put a value on the backorders.

In [15]:

# Optimizing reorder point at central warehouse
# --------------------------------------------------------------------------
# Computing optimal reorder points as well as corresponding expected stock on hand 
# and backorders.
R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdata_df.loc[outdata_df["Type"] == "RDC", "R optimal"] = R_0*Q_subbatch_size
stock_on_hand_wh = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdata_df.loc[outdata_df["Type"] == "RDC","Expected stock on hand"] = stock_on_hand_wh
    
backorders_wh = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdata_df.loc[outdata_df["Type"] == "RDC","Expected backorders"] = backorders_wh

print(f"Optimal (R,Q)-policy at RDC is R = {R_0} and Q = {Q_0} ")

2024-10-15 15:38:50,634 - default_log - DEBUG - new_function
2024-10-15 15:38:50,635 - sparse_log - DEBUG - warehouse_optimization
2024-10-15 15:38:50,636 - default_log - DEBUG - total_cost
2024-10-15 15:38:50,636 - default_log - DEBUG - positive_inventory
2024-10-15 15:38:50,637 - default_log - DEBUG - positive_inventory ended.
2024-10-15 15:38:50,637 - default_log - DEBUG - negative_inventory
2024-10-15 15:38:50,638 - default_log - DEBUG - negative_inventory ended.
2024-10-15 15:38:50,640 - default_log - DEBUG - total_cost ended.
2024-10-15 15:38:50,641 - default_log - DEBUG - total_cost
2024-10-15 15:38:50,641 - default_log - DEBUG - positive_inventory
2024-10-15 15:38:50,643 - default_log - DEBUG - positive_inventory ended.
2024-10-15 15:38:50,644 - default_log - DEBUG - negative_inventory
2024-10-15 15:38:50,645 - default_log - DEBUG - negative_inventory ended.
2024-10-15 15:38:50,646 - default_log - DEBUG - total_cost ended.
2024-10-15 15:38:50,648 - default_log - DEBUG - total_c

Optimal (R,Q)-policy at RDC is R = 65 and Q = 29 


## Computing delay- and lead time
With the optimal policy in place at the RDC, it is time to compute the actual lead time between RDC and dealers depending on the transportation time between installaitons and the delay due to stockouts at the RDC.


In [17]:

# Computing expected delay and lead time
# --------------------------------------------------------------------------
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdata_df.loc[outdata_df["Type"]== "Dealer", "Expected delay"] = W
lead_time_dealer_arr = outdata_df.get(outdata_df["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdata_df.loc[outdata_df["Type"] == "Dealer", "Lead time"] = lead_time_dealer_arr

# Entering dealer lead time demand and standard deviation.
mu_L_dealer_array = outdata_df.get(outdata_df["Type"] == "Dealer").get("Lead time").to_numpy()*outdata_df.get(
        outdata_df["Type"] == "Dealer").get("Demand mean per time unit").to_numpy()
outdata_df.loc[outdata_df["Type"] == "Dealer", "Lead time demand mean"] = mu_L_dealer_array
    
sqrt_dealer_lead_time_arr = np.sqrt(lead_time_dealer_arr)
outdata_df.loc[outdata_df["Type"] == "Dealer", "Lead time demand stdev"] = sqrt_dealer_lead_time_arr*outdata_df.get(outdata_df["Type"]== "Dealer").get("Demand stdev per time unit").to_numpy()
    
# Entering central warehouse lead time as transport time for completeness.
outdata_df.loc[outdata_df["Type"] == "RDC", "Lead time"] = indata_df.get(indata_df["Type"] == "RDC").get("Transport time")

# Computing MTBA (mean time between arrivals) for use in simulation.
# reference: Axsäter, 2006, Inventory control, eq. 5.4
MTBA_arr = np.zeros_like(mu_L_dealer_array)
for i,mu in enumerate(mu_L_dealer_array):
    compounding_dist_arr = compounding_dist_matrix[i]
    j_arr = np.arange(start=1,stop=len(compounding_dist_arr)+1)
    lam = mu/j_arr.dot(compounding_dist_arr)
    MTBA_arr[i] = 1/lam*lead_time_dealer_arr[i]
outdata_df.loc[outdata_df["Type"]== "Dealer", "MTBA"] = MTBA_arr


2024-10-15 15:39:04,713 - default_log - DEBUG - negative_inventory
2024-10-15 15:39:04,714 - default_log - DEBUG - negative_inventory ended.


## Optimizing reorder points at the dealer
Finally, it is time to find optimal (R,Q)-policies at the dealer by minimizing the holding costs under target service level constraints. The actual optimization is done in "dealer_R_optimization()"


In [18]:

# Optimizing reorder points at dealer.
# --------------------------------------------------------------------------
# Computing optimal reorder point, expected realised fill rate, and expected 
# stock on hand level.
opt_dealer_list = []
for Q,L_est,fill_rate_target,demand_type,mu,sigma,compounding_dist_arr in zip(Q_dealer_arr,
    lead_time_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr, compounding_dist_matrix):
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,
        mu,demand_variance = math.pow(sigma,2),compounding_dist_arr=compounding_dist_arr))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[1])
    exp_stock_on_hand_list.append(tup[2])
R_opt_dealer_arr = np.array(R_opt_dealer_list)
fill_rate_dealer_arr = np.array(fill_rate_dealer_list)
exp_stock_on_hand_arr = np.array(exp_stock_on_hand_list)
    

outdata_df.loc[outdata_df["Type"] == "Dealer", "R optimal"] = R_opt_dealer_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_arr
    

# Adding expected backorders at retailers.
exp_backorders_dealer_arr = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdata_df.get(outdata_df["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]
outdata_df.loc[outdata_df["Type"] == "Dealer", "Expected backorders"] = exp_backorders_dealer_arr


## Last touches and output

In [19]:

# Computing cost expressions
# --------------------------------------------------------------------------
total_holding_cost_dealers_arr = h_dealer_arr*exp_stock_on_hand_arr
total_backorder_cost_dealers_arr = exp_backorders_dealer_arr*p_dealer_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Expected holding costs per time unit"] = total_holding_cost_dealers_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Expected shortage costs per time unit"] = total_backorder_cost_dealers_arr
outdata_df.loc[outdata_df["Type"] == "Dealer", "Total expected costs"] = total_holding_cost_dealers_arr + total_backorder_cost_dealers_arr
    
outdata_df.loc[outdata_df["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*stock_on_hand_wh
outdata_df.loc[outdata_df["Type"] == "RDC", "Total expected costs"] = h_rdc*stock_on_hand_wh


In [20]:
# Printing results to CSV.
# --------------------------------------------------------------------------

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

if outdata_path is not None:
    outdata_df.to_csv(outdata_path, index = False)

outdataDF = pd.read_csv(outdata_path, index_col = False)
outdataDF


,Unnamed: 0.1,Unnamed: 0,Installation id,Type,Name,Transport time,Q,Unit cost,Target item fill rate,Demand distribution,Demand mean per time unit,Demand stdev per time unit,Demand type,Stocked,Inventory policy,Lead time demand mean,Lead time demand stdev,Holding cost,Estimated shortage cost,Beta,R optimal,Expected stock on hand,Expected backorders,Expected delay,Lead time,MTBA,Realized item fill rate,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,0,0,A,Dealer,Bartlett,5,9.0,432.90,0.970000,Empiric_Compound_Poisson,0.380267,0.641949,Fast,Yes,BABZA__IP,2.914721,1.777276,0.177904,5.752233,0.151966,13.0,15.136852,0.051573,2.664939,7.664939,3.391619,0.972052,2.692908,0.296659,2.989567
1,1,1,B,Dealer,Bloemfontein,5,3.0,432.90,0.980000,Empiric_Compound_Poisson,0.082613,0.264094,Fast,Yes,BABZA__IP,0.633226,0.731160,0.177904,8.717301,0.276961,2.0,3.368981,0.002207,2.664939,7.664939,12.306327,0.987767,0.599356,0.019238,0.618594
2,2,2,C,Dealer,Capetown,5,4.0,432.90,0.980000,Empiric_Compound_Poisson,0.132640,0.578724,Fast,Yes,BABZA__IP,1.016677,1.602233,0.177904,8.717301,0.427423,4.0,5.491651,0.008329,2.664939,7.664939,7.908773,0.984468,0.976987,0.072605,1.049592
3,3,3,D,Dealer,Durban,5,4.0,432.90,0.980000,Empiric_Compound_Poisson,0.112693,0.463830,Fast,Yes,BABZA__IP,0.863788,1.284142,0.177904,8.717301,0.368602,3.0,4.639366,0.003154,2.664939,7.664939,9.221625,0.990085,0.825362,0.027491,0.852853
4,4,4,E,Dealer,George,10,2.0,432.90,0.985000,Empiric_Compound_Poisson,0.023467,0.153782,Fast,Yes,BABZA__IP,0.297204,0.547278,0.177904,11.682370,0.425677,2.0,3.202940,0.000144,2.664939,12.664939,42.613636,0.998118,0.569816,0.001684,0.571501
5,5,5,F,Dealer,Kimberley,10,1.0,432.90,0.000000,Empiric_Compound_Poisson,0.009090,0.085280,Slow,No,BABZA__IP,0.115124,0.303494,0.177904,0.000000,0.000000,-1.0,0.000000,0.115124,2.664939,12.664939,126.935771,0.000000,0.000000,0.000000,0.000000
6,6,6,G,Dealer,Middelburg,10,6.0,432.90,0.980000,Empiric_Compound_Poisson,0.143733,0.649270,Erratic,Yes,BABZA__IP,1.820374,2.310612,0.177904,8.717301,0.279197,5.0,6.688219,0.008593,2.664939,12.664939,7.706579,0.985239,1.189862,0.074906,1.264767
7,7,7,H,Dealer,Nelspruit,10,3.0,432.90,0.980000,Empiric_Compound_Poisson,0.041333,0.192798,Fast,Yes,BABZA__IP,0.523484,0.686128,0.177904,8.717301,0.285758,3.0,4.478213,0.001697,2.664939,12.664939,29.692082,0.991066,0.796693,0.014798,0.811490
8,8,8,I,Dealer,Port Elizabeth,10,3.0,432.90,0.980000,Empiric_Compound_Poisson,0.060640,0.317588,Fast,Yes,BABZA__IP,0.768002,1.130226,0.177904,8.717301,0.320848,3.0,4.233091,0.001093,2.664939,12.664939,17.336445,0.994433,0.753084,0.009528,0.762612
9,9,9,J,Dealer,Richards Bay,10,3.0,432.90,0.980000,Empiric_Compound_Poisson,0.048107,0.106715,Fast,Yes,BABZA__IP,0.609268,0.379774,0.177904,8.717301,0.135898,2.0,3.392242,0.001510,2.664939,12.664939,20.787140,0.990664,0.603494,0.013159,0.616653
